In [1]:
%matplotlib inline 

import numpy as np
import os
from matplotlib import pyplot as plt

#loading the images
import skimage.io as io



In [2]:
import random
from skimage.transform import resize

fns = os.listdir('MNIST/3')

imgs = []

for i in [3, 8]:
    for fn in os.listdir('MNIST/' + str(i)):
        img = io.imread('MNIST/%d/%s' % (i, fn), dtype=np.float64)
        img = resize(img, (29, 29))
        imgs.append((img, i))

random.shuffle(imgs)
print(imgs[3])

/home/rafael/mestrado/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(array([[2.89744352e+00, 1.52497027e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.18549346e+00, 4.69351962e+00, 3.45659929e+00,
        8.64149822e-01, 0.00000000e+00, 1.25386445e+00, 1.96551724e+00,
        2.52467301e+00, 3.47354340e+00, 4.42241379e+00, 4.45630202e+00,
        2.23662307e+00, 7.79429251e-01, 2.32134364e+00, 2.94827586e+00,
        2.66022592e+00, 1.45719382e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.52497027e-01, 2.70570749e+00, 8.91052319e+00, 9.48275862e+00,
        3.87485137e+00, 7.07193817e-01, 2.47027348e-01, 1.81926278e-01,
        4.06747919e+00, 5.68965517e+00, 3.94084423e+00, 1.23156956e+00,
        1.32877527e-01, 1.82818074e-01, 2.32758621e-01, 2.34542212e-01,
        1.17717004e-01, 4.10225922e-02, 1.22175981e-01, 1.55172414e-01,
        1.40011891e-01, 7.66944114e-02, 2.12544590e-01, 1.12812128e+00,
        1.60225922e+00, 0.00000000e+00

In [3]:
import math
import scipy.ndimage.filters

def convolve(mat, filter): 
    res = scipy.ndimage.filters.convolve(mat, filter)
    (dline, dcol) = (filter.shape[0] - 1) // 2, (filter.shape[1] - 1) // 2
    return res[dline : -dline, dcol : -dcol]
        
def sigmoid(x):
    try:
        return 1 / (1 + math.exp(-x))
    except OverflowError:
        return 1.0 if x > 0 else 0.0

In [14]:
def grad_filter2(factor, mat_inter):
    a = mat_inter.copy()
    return a * factor

def grad_filter1(mat, ginter): 
    a = np.zeros(ginter.shape)
    
    dline, dcol = (a.shape[0] - 1) / 2, (a.shape[1] - 1) / 2
    
    for w_i in np.ndindex(a.shape): 
        for i in np.ndindex(a.shape):
            original_i = (i[0] + w_i[0], i[1] + w_i[1])
            a[w_i] += ginter[i] * mat[original_i]
            
    return a
        
def grad_mat_inter(factor, filter2):
    a = filter2.copy()
    return a * factor

def train_step(mat, filter1, filter2, correct):
    mat_inter = convolve(mat, filter1)
    result_before_sig = convolve(mat_inter, filter2)[0,0]
    result = sigmoid(result_before_sig)
    
#     print('result_before_sig: ', result_before_sig, 'result:', result, 'correct:', correct)
    
    factor = (result - correct) * result * (1 - result)
    
    #print('factor: ', factor )
    
    gf2 = grad_filter2(factor, mat_inter)
    ginter = grad_mat_inter(factor, mat_inter)
    gf1 = grad_filter1(mat, ginter)
    
    lr = 0.00000001
    
    for i in np.ndindex(filter1.shape): 
        #print(lr * gf1[i])
        filter1[i] += lr * gf1[i]
        filter2[i] += lr * gf2[i]
    
    pred = 1 if result > 0.5 else 0
    return pred == correct

In [ ]:
from IPython.display import clear_output

# mat = imgs[3][10]
filter1 = np.random.rand(15,15) * 0.00001
filter2 = np.random.rand(15,15) * 2 - 1


#we are identifying if we obtain an 8
for a in range(100):
    right = 0
    
    for (i, (img, number)) in enumerate(imgs):
        res = train_step(img, filter1, filter2, 1 if number == 8 else 0)
        if res: right += 1
        clear_output(wait=True)
        print(i, '=', right / (i+1) * 100)



1602 = 48.22208359326263
